In [ ]:
import gradio as gr
import random
import time

# --- SAMPLE TRAINING DATA ---
# A small corpus to build our probabilistic model from.
corpus = """
    In the heart of a dense, ancient forest, where sunlight struggled to pierce the thick canopy, lived a creature of myth and legend.
    This was no ordinary beast, but a griffin, with the body of a lion and the head and wings of an eagle.
    Its name was Ignis, and its feathers shimmered with the colors of a sunset.
    Ignis was a guardian, a protector of the forest's deepest secrets.
    One of these secrets was the Moonpetal flower, a bloom that only opened under the light of a full moon and held the power to heal any ailment.
    Many had sought the Moonpetal, but the forest's winding paths and the griffin's watchful eyes kept it safe.
    A young herbalist named Elara, however, was different. She came not with greed, but with a desperate plea.
    Her village was struck by a mysterious illness, and the Moonpetal was their only hope.
    She journeyed for days, her resolve unwavering. When she finally reached the forest's heart, she did not draw a weapon.
    Instead, she offered a simple songbird's feather, a token of peace.
    Ignis, who had seen countless intruders, was intrigued. The griffin listened as Elara spoke of her village's plight.
    Moved by her sincerity, Ignis decided to trust her. The majestic creature led her to a hidden grove where the Moonpetals glowed with an ethereal light.
    Elara took only a single bloom, thanking the griffin with a promise to protect the secret.
    She returned to her village, and the flower's magic worked. The illness vanished.
    The legend of the girl and the griffin became a whispered tale, a reminder that courage and compassion can be more powerful than any sword.
""".strip().lower()

# --- GLOBAL VARIABLES FOR THE MODEL ---
char_to_index = {}
index_to_char = {}
vocab_size = 0
transitions = {}

# --- MODELING LOGIC ---

def build_model():
    """Preprocesses the corpus and builds the transition probability model."""
    global char_to_index, index_to_char, vocab_size, transitions

    chars = sorted(list(set(corpus)))
    vocab_size = len(chars)
    char_to_index = {ch: i for i, ch in enumerate(chars)}
    index_to_char = {i: ch for i, ch in enumerate(chars)}

    # Build a frequency map of next characters
    freq_map = {}
    for i in range(len(corpus) - 1):
        char = corpus[i]
        next_char = corpus[i + 1]
        if char not in freq_map:
            freq_map[char] = {}
        if next_char not in freq_map[char]:
            freq_map[char][next_char] = 0
        freq_map[char][next_char] += 1

    # Convert frequencies to probabilities
    transitions = {}
    for char, next_chars in freq_map.items():
        transitions[char] = {}
        total = sum(next_chars.values())
        for next_char, count in next_chars.items():
            transitions[char][next_char] = count / total

def sample(prob_dist):
    """
    Predicts the next character based on a probability distribution.
    prob_dist is a dictionary like {'a': 0.5, 'b': 0.5}
    """
    rand = random.random()
    cumulative_prob = 0
    # Sort items to ensure consistent order for sampling
    for char, prob in sorted(prob_dist.items()):
        cumulative_prob += prob
        if rand < cumulative_prob:
            return char
    return sorted(prob_dist.keys())[-1] # Fallback

def generate_text(model_type, seed_text, length):
    """
    Simulates text generation using either RNN or LSTM logic.
    """
    if not seed_text:
        return "Error: Please provide some seed text to start."

    generated_text = seed_text
    context = seed_text.lower()

    for _ in range(int(length)):
        # --- MODEL LOGIC SIMULATION ---
        if model_type == 'Simple RNN':
            # Simple RNN: Memory is short, only considers the last character.
            context_len = 1
            recent_context = context[-context_len:]
            last_char = recent_context[-1]
        else: # LSTM
            # LSTM: Simulates longer memory by looking for a valid context char
            # in a larger window.
            context_len = 10
            recent_context = context[-context_len:]
            # Find the most relevant character from the recent context that exists in our model
            last_char = next((char for char in reversed(recent_context) if char in transitions), context[-1])

        # Get the probability distribution for the next character
        next_char_probs = transitions.get(last_char)

        if not next_char_probs:
            # If we have no data for this char, pick a random one from our vocabulary
            next_char = random.choice(list(char_to_index.keys()))
        else:
            next_char = sample(next_char_probs)

        generated_text += next_char
        context += next_char

    return generated_text

# --- BUILD THE MODEL ON STARTUP ---
build_model()
print("Character-level model built from corpus.")

# --- GRADIO INTERFACE ---
with gr.Blocks(theme=gr.themes.Glass(), title="Text Generation") as demo:
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1 style="color: white;">Character-Level Text Generation ✍️</h1>
            <p style="color: #E5E7EB;">Generate text using a simple RNN or an LSTM model. Select a model, provide seed text, and click Generate.</p>
        </div>
        """
    )
    with gr.Row():
        with gr.Column(scale=1):
            model_type_input = gr.Radio(
                ["Simple RNN", "LSTM"],
                label="Select Model",
                value="LSTM"
            )
            seed_text_input = gr.Textbox(
                label="Seed Text",
                value="The ancient forest",
                lines=3,
                placeholder="Enter your starting text here..."
            )
            length_input = gr.Slider(
                50, 1000,
                value=300,
                step=10,
                label="Length of Generated Text"
            )
            generate_btn = gr.Button("Generate Text", variant="primary")

        with gr.Column(scale=2):
            output_text = gr.Textbox(
                label="Generated Output",
                lines=16,
                placeholder="Your generated text will appear here..."
            )

    generate_btn.click(
        fn=generate_text,
        inputs=[model_type_input, seed_text_input, length_input],
        outputs=output_text
    )

if __name__ == "__main__":
    demo.launch()

Character-level model built from corpus.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://388e26b81269bcbb8c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
